File Structure as follows on Google Collab:



*   content/
  *   shape_predictor_68_face_landmarks.dat
  *   {48 photos from 5025 dataset}



Implementation heavily influenced from Adrian Rosebrock's blog, Facial landmarks with dlib, OpenCV, and Python:
https://www.pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/

In [ ]:
#Import necessary packages
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
from google.colab.patches import cv2_imshow
import cv2
import os
import json

In [ ]:
#Download 68 point predictor pre-trained with i-BUG Dataset
!wget -nd https://github.com/JeffTrain/selfie/raw/master/shape_predictor_68_face_landmarks.dat

In [ ]:
dlib_annotation={}
#Instantiate the HOG detector from the DLIB library
detector = dlib.get_frontal_face_detector()
#Instantiate the dlib predictor with the 68 point predictor
shape_predictor = "/content/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(shape_predictor)
#Iterate through images extracting ID from label
for filename in os.listdir("/content/"):
  if filename.endswith(".jpg"):
    if (filename.split("_")[0][0] == '0'):
      id = int(filename.split("_")[0][1])
    else:
      id = int(filename.split("_")[0])

    bb={}
    lm={}

    #Import image in BGR colour mode and convert to grayscale image
    image = cv2.imread("/content/"+filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #Call the HOG face detector to output bounding boxes for the image 
    rects = detector(gray, 1)
    #Call the 68 point predictor on the image and its bounding box
    shape = predictor(gray, rects[0])
    #Covert shape object to numpy array for easier manipulation
    shape = face_utils.shape_to_np(shape)

    #Calculate left eye landmark using the point between a line connecting the edge eye landmarks
    left_eye_x = (float(shape[36][0]) + float(shape[39][0]))/2
    left_eye_y = (float(shape[36][1]) + float(shape[39][1]))/2
    left_eye = [left_eye_x,left_eye_y]
    #Calculate right eye landmark using the point between a line connecting the edge eye landmarks
    right_eye_x = (float(shape[42][0]) + float(shape[45][0]))/2
    right_eye_y = (float(shape[42][1]) + float(shape[45][1]))/2
    right_eye = [right_eye_x,right_eye_y]
    #Extract nose landmark from 68 points
    nose = [float(shape[30][0]),float(shape[30][1])]
    #Extract mouth landmarks from
    left_mouth = [float(shape[48][0]),float(shape[48][1])]
    right_mouth = [float(shape[54][0]),float(shape[54][1])]
    #Extract bounding box values from rect object
    (x, y, w, h) = face_utils.rect_to_bb(rects[0])
    #Show bounding box and 68 point landmark coordinates on face
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    for (x, y) in shape:
      cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
    cv2_imshow(image)
    #Add bounding box and landmark coordinates to dictionary for every face id
    bb = {"x":float(x),"y":float(y),"w":float(w),"h":float(h)}
    lm ={'left_eye': left_eye, 'right_eye': right_eye, 'nose': nose, 'left_mouth': left_mouth, 'right_mouth': right_mouth}
    dlib_annotation[id]={"bounding_box":bb,"landmarks":lm}




In [ ]:
#Export dictionary as JSON file for further analysis
json = json.dumps(dlib_annotation)
f = open("dlib_annotated.json","w")
f.write(json)
f.close()